In [1]:
import os
import pandas as pd
import numpy as np

from data_mani.utils import merge_market_and_gtrends
from prediction.models import RandomForestWrapper
from prediction.functions import get_selected_features, get_features_granger
from prediction.functions import add_shift, annualy_fit_and_predict

In [2]:
ticker_name = "SPX Utilities"
market_folder = "toy"
path_list=["data", "gtrends.csv"]
path_t_list = [path_list[0],
               "src", "data",
               market_folder,
               "{}.csv".format(ticker_name)]
ticker_path = os.path.join(*path_t_list)
ticker_path = os.path.join(*["data",
                            market_folder,
                            "{}.csv".format(ticker_name)])

max_lag = 20
verbose = True
target_name = "target_return"

In [3]:
train, test = merge_market_and_gtrends(ticker_path, test_size=0.5)
words = train.drop(target_name, 1).columns.to_list()
complete = pd.concat([train, test])
del train, test
add_shift(merged_df=complete,
              words=words,
              max_lag=max_lag,
              verbose=verbose)
complete = complete.fillna(0.0)
all_features = complete.drop(words + [target_name], 1).columns.to_list()

add shift: 100%|██████████| 182/182 [00:24<00:00,  7.32it/s]


In [5]:
market_folder = "indices"
fs_method = "sfi"
fs_method = "mdi"
fs_method = "mda"
select = get_selected_features(ticker_name=ticker_name,
                               out_folder="indices",
                               fs_method=fs_method,
                               path_list=path_list)
complete_selected = complete[[target_name] + select]
assert 2 < complete_selected.shape[1] < 3641

In [6]:
Wrapper = RandomForestWrapper
n_iter = 1
n_jobs = 2
n_splits = 2
pred_results = annualy_fit_and_predict(df=complete_selected,
                                       Wrapper=Wrapper,
                                       n_iter=n_iter,
                                       n_jobs=n_jobs,
                                       n_splits=n_splits,
                                       target_name=target_name,
                                       verbose=False)

assert pred_results.shape[0] == complete_selected.loc["2005":].shape[0]